Precision and recall
https://insidelearningmachines.com/precisionk_and_recallk/#:~:text=Precision%40k%20and%20Recall%40k%20are%20metrics%20used%20to%20evaluate,end%20user%20by%20the%20model.

In [ ]:
###precision###

In [ ]:
###recall###

In [ ]:
###F1###

Intra-list Similarity and others https://github.com/statisticianinstilettos/recmetrics